In [23]:
with open("./released/sents.train") as f:
    texts_train = f.read()
f.close()


In [24]:
# words_train = texts_train.replace("\n", " ")[:-1].split(sep=" ")


In [25]:
paragraphs_train = texts_train.split("\n")[:-1]


In [26]:
PennTreebankPOS = {"[START]": {}, "[END]": {}}
WordEmission = {}


for paragraph_train in paragraphs_train:
    words_train = paragraph_train.split(" ")
    prev_POS = "[START]"

    for word_train in words_train:
        tag = word_train.split("/")[-1]
        word = word_train[: -(len(tag) + 1)]

        if not tag in PennTreebankPOS.keys():  # Create new state
            PennTreebankPOS[tag] = {}
        if not tag in PennTreebankPOS[prev_POS].keys():  # a_ij from 0 to 1
            PennTreebankPOS[prev_POS][tag] = 1
        else:
            PennTreebankPOS[prev_POS][tag] += 1  # a_ij from x to x+1 (x > 0)

        # Observation Likelihood
        if not tag in WordEmission.keys():
            WordEmission[tag] = {word: 1}
        else:
            if not word in WordEmission[tag].keys():
                WordEmission[tag][word] = 1
            else:
                WordEmission[tag][word] += 1

        prev_POS = tag

    if not "[END]" in PennTreebankPOS[prev_POS].keys():
        PennTreebankPOS[prev_POS]["[END]"] = 1
    else:
        PennTreebankPOS[prev_POS]["[END]"] += 1


In [41]:
Words = set()
for item in WordEmission.values():
    for word in item.keys():
        Words.add(word)


In [53]:
def ProbScale(x: dict, ref: set or dict or list = None, smoothing: str = None):
    out = x
    for i in out.keys():
        sum_i = sum(out[i].values())
        for j in out[i].keys():
            if not smoothing:
                out[i][j] /= sum_i
            elif smoothing == "Witten-Bell":
                out[i][j] /= sum_i + len(out.keys())
        if smoothing == "Witten-Bell":
            for ind in ref:
                if not ind in out[i].keys():
                    out[i][ind] = len(out.keys()) / (
                        len(ref) * (sum_i + len(out.keys()))
                    )
    return out


In [54]:
POS_trans_mat = ProbScale(PennTreebankPOS, WordEmission.keys(), smoothing="Witten-Bell")
obs_emi_mat = ProbScale(WordEmission, Words, smoothing="Witten-Bell")


In [55]:
WordEmission.keys().__len__()


45